In [1]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

KeyboardInterrupt: ignored

In [ ]:
PATH = '/content/gdrive/MyDrive/Colab Notebooks/Email Spam Classifier'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv(PATH + '/smsspamcollection/SMSSpamCollection',sep='\t',names = ['class','message'])

In [ ]:
data.head()

In [ ]:
data.loc[0,'message']

In [ ]:
data['class'].value_counts()

# DATA Preprocessing

In [ ]:
# Removing everything except [A-Za-z] 
import re
def remove_others(text):
  return re.sub('[^A-Za-z]',' ',text)
  
data['message'] = data['message'].apply(remove_others)

In [ ]:
data['message'][0]

In [ ]:
# Lowercasing

data['message'] = data['message'].apply(str.lower)

In [ ]:
# Remove punctuations
import string
def remove_punc(text):
  return text.translate(str.maketrans('','',string.punctuation))
data['message'] = data['message'].apply(remove_punc)

In [ ]:
# Removing Stop Words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
def remove_stopwords(text):
  return ' '.join(word for word in text.split() if word not in stopwords.words('english'))

data['message'] = data['message'].apply(remove_stopwords)

In [ ]:
data['message'][0]

In [ ]:
# Stemming

from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()
def stemming(text):
  return ' '.join([ps.stem(word) for word in text.split()])

data['message'] = data['message'].apply(stemming)

In [ ]:
data['message'][0]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

cv = TfidfVectorizer(max_features = 3000)
X = cv.fit_transform(data['message']).toarray()

In [ ]:
y = np.where(data['class'] == 'spam',1,0)

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,random_state = 42)

In [ ]:
from sklearn.naive_bayes import BernoulliNB

naive_bayes_model = BernoulliNB()
naive_bayes_model.fit(X_train,y_train)

In [ ]:
predictions = naive_bayes_model.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,predictions)

In [ ]:
from sklearn.metrics import accuracy_score,precision_score
accuracy_score(y_test,predictions)

In [ ]:
precision_score(y_test,predictions)

In [ ]:
import joblib
joblib.dump(naive_bayes_model,PATH + '/naive_bayes_model.sav')

In [ ]:
# joblib.load('naive_bayes_model.sav')

In [ ]:
joblib.dump(cv,PATH + '/TfIdfVectorizer.sav')

In [ ]:
data[data['class'] == 'spam']['message'].head(1)

In [ ]:
np.unique(X[0])